In [133]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

root = "~/Airbnb_predictor"
csv_file = root+"/dummy_csv/19_08_08_listings.csv"
n_file = root+"/category_codes/neighbourhoods.csv"

data = pd.read_csv(csv_file)
n_data = pd.read_csv(n_file)

In [134]:
def price_to_int(data):
    data["price"] = data["price"].apply(lambda x: x.replace('$','').replace(',', '').replace('.00', '')).astype("int")
    return data

In [135]:
data = price_to_int(data)

In [136]:
district = dict(zip(n_data["neighbourhood"].tolist(), n_data["district"].tolist()))
district_group = dict(zip(n_data["neighbourhood"].tolist(), n_data["district_group"].tolist()))
district_number = dict(zip(n_data["neighbourhood"].tolist(), n_data["district_number"].tolist()))
categories = ["neighbourhood_cleansed", "host_is_superhost", "property_type", "room_type", "instant_bookable", "cancellation_policy"]

def map_neighbourhoods(data, district_group, district_number, district):
    data["district"] = data["neighbourhood_cleansed"].map(district)
    data["district_group"] = data["neighbourhood_cleansed"].map(district_group)
    data["district_number"] = data["neighbourhood_cleansed"].map(district_number)
    data["district"]
    return data

def cat_to_code(data, root, categories):
    # Convert categorical data (e.g. neighbourhood_cleansed) into numerical codes
    category_codes = {}
    
    for i in categories:
        name = data[i].name
        category_codes[name] = data[i].astype("category").cat.categories # save list of categories
        data[name+"_codes"] = data[i].astype("category").cat.codes # convert to code
    
    for i in category_codes:
        df = pd.DataFrame()
        df[i] = category_codes[i]
        df.to_csv(root+r"/category_codes/"+i+".csv")
    
    return data


In [137]:
data = map_neighbourhoods(data, district_group, district_number, district)
data = cat_to_code(data, root, categories)

In [144]:
data["district_number"].value_counts()

9     4276
8     3274
2     3004
1     1827
4     1682
3     1497
7     1298
5     1229
6     1071
14     703
10     661
15     574
11     233
13     201
12      87
Name: district_number, dtype: int64

In [46]:
properties = ["Apartment", "Condominium", "House", "Townhouse", "Guest suite", "Bungalow"]
can_policy = ["flexible", "moderate", "strict_14_with_grace_period"]

def filter_categorical(properties, can_policy, data):
    num_samples = data.shape[0]
    data = data[data.property_type.isin(properties)]
    num_samples2 = data.shape[0]
    data = data[data.cancellation_policy.isin(can_policy)]
    num_samples3 = data.shape[0]
    removed = {"properties": num_samples-num_samples2,
               "can_policy": num_samples2-num_samples3}
    return data, removed


In [47]:
data, removed = filter_categorical(properties, can_policy, data)

In [54]:
data['neighbourhood_cleansed'].value_counts()

Waterfront Communities-The Island      3718
Niagara                                 880
Annex                                   702
Church-Yonge Corridor                   690
Bay Street Corridor                     616
Dovercourt-Wallace Emerson-Junction     551
Kensington-Chinatown                    539
Willowdale East                         503
Trinity-Bellwoods                       481
Little Portugal                         480
Moss Park                               425
Palmerston-Little Italy                 405
South Riverdale                         350
South Parkdale                          308
Cabbagetown-South St.James Town         307
Mount Pleasant West                     261
Mimico (includes Humber Bay Shores)     254
York University Heights                 240
Roncesvalles                            230
Dufferin Grove                          210
North St.James Town                     203
High Park-Swansea                       201
The Beaches                     

In [ ]:
class airbnb_dataset(Dataset, data_frame):
    def __init__(self):
        self.data_frame = data_frame
        self.shape = data_frame.shape
    
    def __len__(self):
        return len(data_frame.shape)
    
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        
        target = data_frame["price"][index]
        sample = [data_frame]
        
